# Submitting Qiskit Circuits to Azure Quantum with the QDK

This notebook demonstrates two ways to run Qiskit `QuantumCircuit` jobs on Azure Quantum using the QDK: (A) direct submission via `AzureQuantumProvider` for a streamlined workflow, and (B) an explicit OpenQASM 3 → QIR compilation path for transparency and artifact inspection. It also shows how to handle parameterized circuits by binding parameters prior to execution.

This workflow demonstrates two methods of submission:

### A. Direct submission via `AzureQuantumProvider` (recommended when available)
1. Build (or load) a Qiskit `QuantumCircuit`.
2. Reference an existing Azure Quantum workspace with `qdk.azure.Workspace`.
3. Instantiate `AzureQuantumProvider(workspace)` and pick a backend (`provider.get_backend(<target_name>)`).
4. Call `backend.run(circuit, shots, job_name=...)` and fetch results (`job.result().get_counts(circuit)`).

### B. Submit via OpenQASM compilation (explicit OpenQASM → QIR → submit)
1. Build (or load) a Qiskit `QuantumCircuit`.
2. Convert it to OpenQASM 3 text (`qiskit.qasm3.dumps`).
3. Compile the OpenQASM 3 source to QIR with `qdk.openqasm.compile` (choose a `TargetProfile`).
4. Reference an existing Azure Quantum workspace with `qdk.azure.Workspace`.
5. Select a target (e.g. a simulator such as `rigetti.sim.qvm`).
6. Submit the QIR payload (`target.submit(qir, job_name, shots)`), then retrieve results (`job.get_results()`).

## Prerequisites

This notebook assumes the `qdk`, Azure Quantum integration, and Qiskit packages are installed. You can install everything (including the Azure + Qiskit extras) with:

In [ ]:
%pip install "qdk[azure,qiskit]"

This installs:
- The base qdk package (compiler, OpenQASM/QIR tooling)
- Azure Quantum client dependencies for submission
- Qiskit for circuit construction

After installing, restart the notebook kernel if it was already running. You can verify installation with:

In [ ]:
import qiskit, qdk, qdk.azure  # should import without errors

## Submitting a simple Qiskit circuit

We start with a minimal circuit that prepares single-qubit superpositions on two qubits and measures them. After constructing the circuit we’ll submit it to an Azure Quantum target.

In [ ]:
from qiskit import QuantumCircuit

circuit = QuantumCircuit(2, 2)

circuit.h(0)
circuit.measure(0, 0)

circuit.h(1)
circuit.measure(1, 1)

circuit.draw(output="text")

## Configure Azure Quantum workspace connection

To connect to an Azure workspace replace the following variables with your own values.

In [ ]:
subscription_id = 'xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx'
resource_group = 'myresourcegroup'
workspace_name = 'myworkspace'
location = 'westus'

### Approach A: Using Azure Quantum Provider

Here we use the recommended submission method with `AzureQuantumProvider` to submit the Qiskit circuit straight to an Azure Quantum backend. This avoids having to explicitly do QASM translation or QIR compilation steps.

In [ ]:
from qdk.azure import Workspace
from qdk.azure.qiskit import AzureQuantumProvider

def submit_qiskit_circuit_to_azure_provider(circuit, target_name, name, shots=100):

    workspace = Workspace(
        subscription_id=subscription_id,
        resource_group=resource_group,
        name=workspace_name,
        location=location,
    )

    provider = AzureQuantumProvider(workspace)
    backend = provider.get_backend(target_name)
    job = backend.run(circuit, shots, job_name=name)
    return job.result().get_counts(circuit)

provider_counts = submit_qiskit_circuit_to_azure_provider(circuit, "rigetti.sim.qvm", "qiskit-provider-job")
print(provider_counts)

### Approach B: Submit via OpenQASM compilation

Below we show the longer path that exposes intermediate artifacts. This is useful if you want to inspect or transform OpenQASM/QIR, or integrate with tooling that consumes QIR directly.


In [ ]:
from qiskit import qasm3
from qdk.openqasm import compile
from qdk import TargetProfile

def submit_qiskit_circuit_to_azure_via_qasm(circuit, target_name, name, shots=100):
    qasm3_str = qasm3.dumps(circuit)
    qir = compile(qasm3_str, target_profile=TargetProfile.Base)

    workspace = Workspace(
        subscription_id=subscription_id,
        resource_group=resource_group,
        name=workspace_name,
        location=location,
    )
    target = workspace.get_targets(target_name)
    job = target.submit(qir, name, shots=shots)
    return job.get_results()

results = submit_qiskit_circuit_to_azure_via_qasm(circuit, "rigetti.sim.qvm", "qiskit-via-qasm-job")
print(results)

## Parameterized Qiskit circuits

Many algorithms use symbolic parameters. Before submitting to azure (or before compiling to QIR), all circuit parameters must be bound to numeric values. Below we build a simple entangling ladder, apply a rotation parameterized by `θ` across all qubits, then uncompute the entanglement and measure the first qubit.

In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter

def get_parameterized_circuit(n = 5) -> QuantumCircuit:
    theta = Parameter("θ")
    qc = QuantumCircuit(n, 1)
    qc.h(0)
    for i in range(n - 1):
        qc.cx(i, i + 1)
    qc.rz(theta, range(n))

    for i in reversed(range(n - 1)):
        qc.cx(i, i + 1)
    qc.h(0)
    qc.measure(0, 0)
    return qc

# Build the symbolic (parameterized) circuit
parameterized_circuit = get_parameterized_circuit()

# Bind θ to a numeric value, then visualize the bound circuit
bound_circuit = parameterized_circuit.assign_parameters({"θ": 0.5})

bound_circuit.draw(output="text")

## Submitting a bound parameterized circuit

Here we submit the `bound_circuit` from above using the recommended `AzureQuantumProvider` approach. The printed result shows the measurement counts. Change the value of `θ` (or loop over several values) to explore how the outcome distribution varies.

In [ ]:
results = submit_qiskit_circuit_to_azure_provider(
    bound_circuit,
    "rigetti.sim.qvm",
    "qiskit-parameterized-job"
)
print(results)